``pivot_table``은 스프레드시트 프로그램과 다른 데이터 분석 소프트웨어에서 흔히 볼 수 있는 데이터 요약화 도구다.<br/>
`pivot_table`은 데이터를 하나 이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다.<br/>
pandas에서 `pivot_table`은 이 장에서 설명했던 groupby 기능을 사용해서 계층적 색인을 활용한 재형성 연산을 가능하게 해준다.<br/>
DataFrame에는 `pivot_table` 메서드가 있는데, 이는 pandas 모듈의 최상위 함수이기도 하다.(`pandas.pivot_table`)<br/>
groupby를 위한 편리한 인터페이스를 제공하기 위해 `pivot_table`은 ***margin***이라고 하는 부분합을 추가할 수 있는 기능을 제공한다.

In [1]:
from seaborn import load_dataset

In [2]:
tips = load_dataset('tips')

In [4]:
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


계속 살펴보았던 tips 데이터에서 성별과 흡연자 집단의 평균(pivot_table의 기본연산)을 구해보자.

In [19]:
tips.pivot_table(index=['sex','smoker'])

size       tip  total_bill
sex    smoker                                
Male   Yes     2.500000  3.051167   22.284500
       No      2.711340  3.113402   19.791237
Female Yes     2.242424  2.931515   17.977879
       No      2.592593  2.773519   18.105185

In [20]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

In [21]:
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


groupby를 사용해서 쉽게 구할 수 있는데, 이제 tip_pct와 size에 대해서만 집계를 하고 날짜별로 그룹을 지어보자.<br/>
이를 위해 day로우와 smoker 컬럼을 추가했다.

In [23]:
tips.pivot_table(values=['tip_pct','size'], index=['sex','day'], columns='smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

이 표는 `margins=True`를 넘겨서 부분합을 포함하도록 확장할 수 있는데, 그렇게 하면 All 컬럼과 All 로우가 추가되어 단일 줄 안에서 그룹 통계를 얻을 수 있다.<br/>
다음 예제에서는 All 값은 흡연자와 비흡연자를 구분하지 않은 평균 값(All 컬럼)이거나 로우에서 두 단계를 묶은 그룹의 평균 값(All 로우)이다.

In [24]:
tips.pivot_table(values=['tip_pct','size'], index=['sex','day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
sex    day                                                             
Male   Thur  2.300000  2.500000  2.433333  0.164417  0.165706  0.165276
       Fri   2.125000  2.000000  2.100000  0.144730  0.138005  0.143385
       Sat   2.629630  2.656250  2.644068  0.139067  0.162132  0.151577
       Sun   2.600000  2.883721  2.810345  0.173964  0.158291  0.162344
Female Thur  2.428571  2.480000  2.468750  0.163073  0.155971  0.157525
       Fri   2.000000  2.500000  2.111111  0.209129  0.165296  0.199388
       Sat   2.200000  2.307692  2.250000  0.163817  0.147993  0.156470
       Sun   2.500000  3.071429  2.944444  0.237075  0.165710  0.181569
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

다른 집계 함수를 사용하려면 그냥 `aggfunc`에 넘기면 되는데, 한 예로 count나 len 함수는 그룹 크기의 교차일람표(총 개수나 빈도)를 반환한다.

In [25]:
tips.pivot_table(values='tip_pct', index=['sex','smoker'], columns='day', aggfunc=len, margins=True)

day            Thur   Fri   Sat   Sun    All
sex    smoker                               
Male   Yes     10.0   8.0  27.0  15.0   60.0
       No      20.0   2.0  32.0  43.0   97.0
Female Yes      7.0   7.0  15.0   4.0   33.0
       No      25.0   2.0  13.0  14.0   54.0
All            62.0  19.0  87.0  76.0  244.0

만약 어떤 조합이 비어있거나 NA값이라면 fill_value를 넘길 수도 있다.

In [32]:
tips.pivot_table(values='size', index=['time','sex','smoker'], columns='day', aggfunc=sum)

day                   Thur   Fri   Sat    Sun
time   sex    smoker                         
Lunch  Male   Yes     23.0   5.0   NaN    NaN
              No      50.0   NaN   NaN    NaN
       Female Yes     17.0   6.0   NaN    NaN
              No      60.0   3.0   NaN    NaN
Dinner Male   Yes      NaN  12.0  71.0   39.0
              No       NaN   4.0  85.0  124.0
       Female Yes      NaN   8.0  33.0   10.0
              No       2.0   2.0  30.0   43.0

In [31]:
tips.pivot_table(values='size', index=['time','sex','smoker'], columns='day', aggfunc=sum, fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43